In [ ]:
import contextily as cx
import geopandas as gpd
import pandas as pd

from uhi_drivers_lausanne import regr_utils

In [ ]:
# TODO: merge into single ts_df in data/processed?
official_ts_df_filepath = "../data/interim/official-ts-df.csv"
cws_ts_df_filepath = "../data/interim/cws-qc-ts-df.csv"

official_stations_gdf_filepath = "../data/interim/official-stations.gpkg"
cws_stations_gdf_filepath = "../data/interim/cws-qc-stations.gpkg"
stations_crs = "epsg:2056"

dst_station_id_col = "station_id"
dst_stations_gdf_filepath = "../data/processed/stations.gpkg"
dst_ts_df_filepath = "../data/processed/ts-df.csv"

In [ ]:
stations_gdf = pd.concat(
    [
        gpd.read_file(stations_gdf_filepath)
        .to_crs(stations_crs)
        .rename(columns={_station_id_col: dst_station_id_col})[
            [dst_station_id_col, "geometry"]
        ]
        .assign(source=source)
        for stations_gdf_filepath, _station_id_col, source in zip(
            [official_stations_gdf_filepath, cws_stations_gdf_filepath],
            ["index", "station_id"],
            ["official", "cws"],
        )
    ]
).set_index(dst_station_id_col)
# plot the geo-data frame
ax = stations_gdf.plot("source", legend=True)
cx.add_basemap(ax, crs=stations_gdf.crs)

In [ ]:
ts_df = regr_utils.get_long_ts_df(
    [official_ts_df_filepath, cws_ts_df_filepath],
    dst_station_id_col,
)
ts_df["source"] = ts_df[dst_station_id_col].map(stations_gdf["source"].to_dict())
ts_df["UHI"] = pd.concat(
    [
        regr_utils.get_uhi_df(ts_df[ts_df["source"] == source])
        for source in ["official", "cws"]
    ]
)["UHI"]
ts_df

In [ ]:
# dump the data
ts_df.to_csv(dst_ts_df_filepath, index=False)
stations_gdf.to_file(dst_stations_gdf_filepath)